In [1]:
from features import ensembleFeature
import pandas as pd
import numpy as np
from utils import metricsCal
from thundersvm import SVC
from sklearn.model_selection import KFold

In [2]:
##将序列转化为词向量用到的函数
def fa_seq(filepath):
    f = open(filepath,'r')
    x = []
    for i in f:
        x.append(i)
    seq = []
    for i in range(len(x)):
        if i%2==1:
            seq.append(x[i][:-1])
    return seq

In [3]:
train_seq = fa_seq("data/Mouse/mouse_train.fasta")
trainLabel = np.append(np.ones(int(len(train_seq)/2)),np.zeros(int(len(train_seq)-len(train_seq)/2)),axis=0)
trainData_EIIP = ensembleFeature.EIIP(train_seq)

trainData_PseEIIP = ensembleFeature.PseEIIP(train_seq)

trainData_PCP = ensembleFeature.PCP(train_seq)

trainData_NCPA = ensembleFeature.NCPA(train_seq)
trainData_NCPA = trainData_NCPA.reshape(trainData_NCPA.shape[0],-1)

trainData_DBPF = ensembleFeature.DBPF(train_seq)
trainData_DBPF = trainData_DBPF.reshape(trainData_DBPF.shape[0],-1)

trainData = np.concatenate(  (trainData_EIIP,trainData_PseEIIP,trainData_PCP,trainData_NCPA,trainData_DBPF),axis=1)

In [8]:
from sklearn.manifold import TSNE
ts = TSNE(n_components=3, init='pca', random_state=0)
reslut = ts.fit_transform(trainData)

In [13]:
#交叉验证
rows = 10
row = -1
row_name = []
Sen = np.zeros(rows)
Spe = np.zeros(rows)
Acc = np.zeros(rows)
Mcc = np.zeros(rows)
Auc = np.zeros(rows)

kf = KFold(10,True,10)
for i,[train_index, test_index] in enumerate(kf.split(reslut)):
    row+=1
    row_one = "Bert6mA_origin_KFold_" + str(i)
    row_name.append(row_one)
    X_train = reslut[train_index]
    X_test = reslut[test_index]
    Y_train = trainLabel[train_index]
    Y_test = trainLabel[test_index]
    
    model = SVC(C=1.0,gamma="auto")
    
    model.fit(X_train,Y_train)
    th,_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics( model.decision_function(X_train).reshape(-1), Y_train )
    _,_,_,_,sen,spe,acc,mcc,auc = metricsCal.get_test_metrics( model.decision_function(X_test), Y_test,th )
    Sen[row] = sen
    Spe[row] = spe
    Acc[row] = acc
    Mcc[row] = mcc
    Auc[row] = auc
    
All = np.vstack((Sen,Spe,Acc,Mcc,Auc))
np.mean(All[4])

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


0.6237314368529492

In [12]:
reslut

array([[-43.0365    ,   1.3645849 ,  25.712265  ],
       [-31.5757    ,  41.565933  ,  -0.36372098],
       [  4.920306  ,   8.533412  , -36.097176  ],
       ...,
       [ 62.108524  ,  -0.83823633,  20.48054   ],
       [ -3.2842798 , -38.585526  , -13.183204  ],
       [-51.368225  , -28.309515  , -10.514868  ]], dtype=float32)